In [1]:
pip install together

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain_together

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_together import TogetherEmbeddings
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel(r"D:\Srujan\Projects\Infosys\reviews_data.xlsx")

In [5]:
df

,review_id,customer_id,review_date,Review,Rating,review_date_numeric
0,11722,5380,2024-11-04,The room provided at first was a great disapp...,6.7,20241104
1,8604,8832,2024-03-01,The room wasn t so large for a Deluxe King bu...,9.6,20240301
2,3923,7301,2023-09-04,on arrival we had to wait over an hour after ...,7.1,20230904
3,10050,3968,2023-09-02,The room reminded me of a dorm room furnished...,7.9,20230902
4,2077,6085,2024-05-17,The room was very small the AC didn t work at...,8.8,20240517
...,...,...,...,...,...,...
8377,5928,1546,2023-04-30,Bathroom and toilet are small but spotlessly ...,10.0,20230430
8378,7439,4787,2024-01-30,Heating system in the room instructions on ho...,9.2,20240130
8379,339,4936,2024-10-11,Entering the hotel involved crossing an open ...,9.2,20241011
8380,6816,8542,2024-09-01,The hotel is very old Room are out of date an...,4.2,20240901


In [6]:
import os
import getpass
if not os.getenv("TOGETHER_API_KEY"):
    os.environ["TOGETHER_API_KEY"] = "d0138d7876bde3db01c9b87dd4a7fc88b67d130c13d26cb8db4b874cd6feb275"

# Initialize the TogetherEmbeddings model
embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval"  # Example model; check docs for Llama-based alternatives
)

In [7]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


# Create embeddings for all reviews - this will take close to 1.5 - 2 hours

In [8]:
reviews = df["Review"].tolist()

# Process embeddings in batches
embedding_list = []
for i in range(0, len(reviews), 128):
    batch = reviews[i : i + 128]  # Get batch
    batch_embeddings = embeddings.embed_documents(batch)  # Generate embeddings for batch
    embedding_list.extend(batch_embeddings)  # Store results
    print(f"Processed {i + len(batch)} / {len(reviews)} reviews")

Processed 128 / 8382 reviews
Processed 256 / 8382 reviews
Processed 384 / 8382 reviews
Processed 512 / 8382 reviews
Processed 640 / 8382 reviews
Processed 768 / 8382 reviews
Processed 896 / 8382 reviews
Processed 1024 / 8382 reviews
Processed 1152 / 8382 reviews
Processed 1280 / 8382 reviews
Processed 1408 / 8382 reviews
Processed 1536 / 8382 reviews
Processed 1664 / 8382 reviews
Processed 1792 / 8382 reviews
Processed 1920 / 8382 reviews
Processed 2048 / 8382 reviews
Processed 2176 / 8382 reviews
Processed 2304 / 8382 reviews
Processed 2432 / 8382 reviews
Processed 2560 / 8382 reviews
Processed 2688 / 8382 reviews
Processed 2816 / 8382 reviews
Processed 2944 / 8382 reviews
Processed 3072 / 8382 reviews
Processed 3200 / 8382 reviews
Processed 3328 / 8382 reviews
Processed 3456 / 8382 reviews
Processed 3584 / 8382 reviews
Processed 3712 / 8382 reviews
Processed 3840 / 8382 reviews
Processed 3968 / 8382 reviews
Processed 4096 / 8382 reviews
Processed 4224 / 8382 reviews
Processed 4352 / 

In [9]:
metadata_list = df.apply(lambda row: {
    "customer_id": int(row["customer_id"]),
    "review_date": row["review_date_numeric"],
    "Rating": int(row["Rating"]),
    "review_id":row['review_id']
}, axis=1).tolist()

In [10]:
metadata_list[0]

{'customer_id': 5380, 'review_date': 20241104, 'Rating': 6, 'review_id': 11722}

In [16]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key='pcsk_3Akygq_37NwsGLrrUn2yjAhpNymMFUxqh5ggkET7tZHnmfiuyiwvCFWu5a3gRD3mCvp1KX'
)

# Check whether any indexes already exist

In [17]:
pc.list_indexes()

[
    {
        "name": "hotel-reviews",
        "metric": "cosine",
        "host": "hotel-reviews-t1jsbtd.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 768,
        "deletion_protection": "enabled",
        "tags": null
    }
]

# create the index

In [21]:
import pinecone

# Create an instance of the Pinecone class
pc = pinecone.Pinecone(api_key="pcsk_3Akygq_37NwsGLrrUn2yjAhpNymMFUxqh5ggkET7tZHnmfiuyiwvCFWu5a3gRD3mCvp1KX")

# List available indexes
print(pc.list_indexes().names())


['hotel-reviews']


In [ ]:
pc.create_index(
    name='hotel-reviews',
    dimension=768,
    metric='cosine',
    deletion_protection='enabled',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

In [26]:
index = pc.Index(host="hotel-reviews-t1jsbtd.svc.aped-4627-b74a.pinecone.io")


In [27]:
vectors = [
    (str(i), embedding, metadata_list[i])
    for i, embedding in enumerate(embedding_list)
]

# Insert embeddings + metadata into the index

In [28]:
batch_size = 100  # Adjust this number as needed.

for i in range(0, len(embedding_list), batch_size):
    batch_vectors = [
        (str(i + j), embedding_list[i + j], metadata_list[i + j])
        for j in range(min(batch_size, len(embedding_list) - i))
    ]
    index.upsert(vectors=batch_vectors)
    print(f"Upserted batch from {i} to {i + len(batch_vectors)}")

Upserted batch from 0 to 100
Upserted batch from 100 to 200
Upserted batch from 200 to 300
Upserted batch from 300 to 400
Upserted batch from 400 to 500
Upserted batch from 500 to 600
Upserted batch from 600 to 700
Upserted batch from 700 to 800
Upserted batch from 800 to 900
Upserted batch from 900 to 1000
Upserted batch from 1000 to 1100
Upserted batch from 1100 to 1200
Upserted batch from 1200 to 1300
Upserted batch from 1300 to 1400
Upserted batch from 1400 to 1500
Upserted batch from 1500 to 1600
Upserted batch from 1600 to 1700
Upserted batch from 1700 to 1800
Upserted batch from 1800 to 1900
Upserted batch from 1900 to 2000
Upserted batch from 2000 to 2100
Upserted batch from 2100 to 2200
Upserted batch from 2200 to 2300
Upserted batch from 2300 to 2400
Upserted batch from 2400 to 2500
Upserted batch from 2500 to 2600
Upserted batch from 2600 to 2700
Upserted batch from 2700 to 2800
Upserted batch from 2800 to 2900
Upserted batch from 2900 to 3000
Upserted batch from 3000 to 310

# experiment with below parts

In [29]:
query_embedding = embeddings.embed_query("What are some of the reviews that mention restaurant, food, lunch, breakfast, dinner")

In [38]:
results = index.query(
    vector=query_embedding,
    top_k=5,
    namespace="",
    include_metadata=True,
    filter={
        "Rating": {"$lte": 7},
        "review_date": {"$gte": 20240101, "$lte": 20240108}
    }
)

In [39]:
matches = results["matches"]

# Extract review_ids from the matches (convert them to int if necessary)
matched_ids = [int(match["metadata"]["review_id"]) for match in matches]

In [40]:
matched_ids

[11531, 2910, 6390, 8397, 7555]

In [41]:
req_df = df[df["review_id"].isin(matched_ids)]

In [42]:
req_df['Review']

319      Need wait to sitting for breakfast come so ma...
486      Shower taps need descaling horrid smell from ...
1034     I booked a plan that included breakfast When ...
1622     The people working on the terrace bar are pre...
3105     High parking fee When I booked this hotel I r...
Name: Review, dtype: object

In [43]:
concatenated_reviews = " ".join(req_df["Review"].tolist())

In [44]:
concatenated_reviews

' Need wait to sitting for breakfast come so many people and not enaugh place So noisily neighbour in room I mean hotel don t have protection for sound Guests in next room speak and we listen all Special in 1am Can t sleep If you want to go for dinner is too far from hotel   Shower taps need descaling horrid smell from drains the new booth seating is so unfriendly the older decor made more sense New menu card room dining not advisable limited choice of average food Tea cakes were cold from fridge so cream in the cakes was There were no more small tasty affordable snacks like the crab cakes of tandoori salmon or tatty etc Pity You lost 15 customers who went to eat elsewhere daily for 3 days   location  I booked a plan that included breakfast When arrived the lady who checked me and my mother in insisted under the supervision and instructions of another member of staff that we booked a plan without meals included even after showing the email from Booking com stating that breakfast was in

In [45]:
from together import Together
import os
client = Together()
response = client.chat.completions.create(
    model="meta-llama/Llama-Vision-Free",
    messages = [{"role": "user", "content": f"""Briefly Summarize the overall sentiment of customers about food and restaurant based on these reviews - {
    concatenated_reviews}. Dont mention the name of the hotel"""}]
)
print(response.choices[0].message.content)


Here is a brief summary of the overall sentiment of customers about food and restaurant at the hotel:

**Negative comments:**

* Limited and unimpressive food options
* Noise and lack of sound protection in rooms
* Long wait times for food and drinks
* Limited and unaffordable snack options
* Poor service, including mistakes with orders and billing
* Unfriendly new seating area
* Limited and average food quality
* Tea cakes were cold and cream was spoiled

**Positive comments:**

* Location was perfect for sightseeing and enjoying good restaurants
* Hotel was comfortable and quiet
* Terrace bar had a great view of Barcelona
* Some guests enjoyed their stay and would recommend the hotel for its location.

Overall, it seems that the hotel's restaurant and food services were a major source of frustration for many guests, with complaints about long wait times, poor service, and unimpressive food options. However, some guests appreciated the location and comfortable atmosphere of the hotel.